# Focus Radargram
## 0. Imports

In [ ]:
from surface import *
from source import *
from model import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from math import floor
from math import ceil

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## 1. Load radargram

In [ ]:
rdrgrm = np.load("rdrgrm.npy")

In [ ]:
# radargram params
st_t = 166.8e-6   # start
en_t = 175e-6  # end
N = rdrgrm.shape[0]      # how many "range bins?"
n = rdrgrm.shape[1]      # how many traces
t_bin = (en_t-st_t) / N

In [ ]:
fig = px.imshow(rdrgrm, aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))

fig.update_yaxes(
    tickvals=tickvals,
    ticktext=ticktext,
    title_text="Time (µs)"
)
fig.show()

## 2. Range Compression

In [ ]:
from scipy.fft import fft, ifft

In [ ]:
rc = np.zeros_like(rdrgrm)

# generate sample source to cross correlate with
source = Source(1e-9, 1.0e-6, (0, 0, 0))
source.chirp(9e6, 1e6, 0.5e-6)
signal = source.signal

# pad signal to be same length as N
signal = np.concatenate((np.zeros(int(1+(N-len(signal))/2)),
                         signal,
                         np.zeros(int((N-len(signal))/2))))

# compute fft of source
fft_source = fft(signal)

for i in range(n):
    fft_trace = fft(rdrgrm[:, i])
    cc_freq = fft_source * np.conj(fft_trace)
    cc_time = ifft(cc_freq)
    cc_time = np.fft.fftshift(cc_time)
    rc[:,i] = np.flip(np.real(cc_time))

In [ ]:
fig = px.imshow(rc, aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
fig.update_xaxes(title_text="Trace #")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 3. Azumith FFT & Delay Estimation

In [ ]:
az_fft = np.fft.fft(rc, axis=1)
az_freq = np.fft.fftshift(np.fft.fftfreq(n, d=source.dt))
az_fftshift = np.real(np.fft.fftshift(az_fft, axes=1))

In [ ]:
# delay estimation
az_max = np.argmax(az_fftshift, axis=0)

In [ ]:
fig = px.imshow(az_fftshift, aspect="auto", color_continuous_scale='gray', 
                zmin=0, zmax=1000, width=800, height=600)

ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
axtext = ['{0:1f}'.format(v/(1e6)) for v in np.interp(np.linspace(0, n, 21), range(n), az_freq)]
axvals = np.linspace(0, n, len(axtext))

fig.add_trace(go.Scatter(
    x=list(range(n)), 
    y=az_max,
    mode='lines',
    name='Delay Estimate',
    line=dict(color='red')
))

fig.update_xaxes(tickvals=axvals,   ticktext=axtext,   title_text="Doppler Domain - Freq (MHz)")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 4. PF RCMC (Point Facet Range Cell Migration Correction)

In [ ]:
timebuff = int(az_fftshift.shape[0]/4)
rcmc = np.zeros((int(2*timebuff), n))
for i, az in enumerate(az_max):
    tmin, tmax = max(az-timebuff, 0), min(az_fftshift.shape[0], az+timebuff)
    segment = az_fftshift[tmin:tmax, i]
    rcmc[:len(segment), i] = segment

In [ ]:
fig = px.imshow(rcmc, aspect="auto", color_continuous_scale='gray', 
                zmin=0, zmax=1000, width=800, height=300)
r1 = timebuff / az_fftshift.shape[0]
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(r1*(en_t-st_t)*-1e6, (1-r1)*(en_t-st_t)*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
axtext = ['{0:1f}'.format(v/(1e6)) for v in np.interp(np.linspace(0, n, 21), range(n), az_freq)]
axvals = np.linspace(0, n, len(axtext))

fig.add_trace(go.Scatter(
    x=list(range(n)), 
    y=np.zeros(n)+timebuff,
    mode='lines',
    name='Delay Estimate',
    line=dict(color='red')
))

fig.update_xaxes(tickvals=axvals,   ticktext=axtext,   title_text="Doppler Domain - Freq (MHz)")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 5. Extract 1D Reference Function

In [ ]:
reffun = rcmc[timebuff, :]

In [ ]:
df = pd.DataFrame({'Doppler Domain - Freq (Hz)':np.interp(np.linspace(0, n, n), range(n), az_freq), 'Power (dB)':10*np.log10(reffun)})
fig = px.line(df, x='Doppler Domain - Freq (Hz)', y='Power (dB)')
fig.show()

## 6. Azumith Compression

In [ ]:
azcmp = np.zeros_like(rcmc)

# compute fft of reference function
fft_source = fft(reffun)

for i in range(rcmc.shape[0]):
    fft_trace = fft(rcmc[i, :])
    cc_freq = fft_source * np.conj(fft_trace)
    cc_time = ifft(cc_freq)
    cc_time = np.fft.fftshift(cc_time)
    azcmp[i,:] = np.flip(np.real(cc_time))

## 7. Azumith IFFT

In [ ]:
azcmp_shift = np.fft.ifftshift(azcmp)
focused = np.fft.ifft(azcmp_shift, axis=1)

In [ ]:
fig = px.imshow(np.abs(focused), aspect="auto", color_continuous_scale='gray', width=800, height=600, zmin=0, zmax=5e6)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
fig.update_xaxes(title_text="Trace #")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 3. Develop reference echo response

In [ ]:
# make surface
surf = Surface(origin=(0, 0), dims=(101, 101), fs=100)
surf.gen_sin('x', 50, 1000, -10)

In [ ]:
models = []
en = 10000
for x in np.linspace(0, en, n):
    print(f"Generating: {round(x)}/{en} ({round(100*(x/en), 1)}%)", end="     \r")
    # make source
    source = Source(1e-9, 0.25e-6, (x, 5050, 25000))
    _, _ = source.chirp(9e6, 1e6)
    # make model
    model = Model(surf, source)
    model.set_target((5050, 5050, -200))
    model.gen_raypaths()
    models.append(model)
models = np.array(models)

In [ ]:
conjs = np.array([m.ref_funct_conj(None, None) for m in models])
plt.plot(range(len(conjs)), np.real(conjs), label="real")
plt.plot(range(len(conjs)), np.imag(conjs), label="imag")
plt.xlabel("Sample #")
plt.ylabel("Reference echo response")
plt.show()

## 4. 1D Focus

In [ ]:
def focus_pixel_1D(x, t_m, L1D, conj):
    
    dx = L1D / n
    
    c_i = int(x / dx)
    l_i = max((int((x-(L1D/2))/dx), 0))
    r_i = min((int((x+(L1D/2))/dx), n))
    conj_set = conj[l_i:r_i]
    
    j = int((t_m-st_t)/t_bin)
    s = rdrgrm[j-1, l_i:r_i]
    
    return np.sum(np.real(conj_set) * s * dx)

In [ ]:
from time import time as Time

In [ ]:
L1D = 10000
T = 0.25e-6

focused_1d = np.zeros_like(rdrgrm)

tst = Time()
for i, x in enumerate(np.linspace(0, en, n)):
    tnow = Time()
    print(f"Focusing: {i+1}/{n} ({round(100*(i+1)/(n), 2)}%) Runtime: {round((tnow-tst)/60, 2)} min Remaining: {round(((n)/(i+1))*(tnow-tst)/60, 2)} min", end="\t\t\r")
    for j, t_m in enumerate(np.linspace(st_t, en_t, N)):
        focused_1d[j, i] = focus_pixel_1D(x, t_m, L1D, conjs)

In [ ]:
plt.imshow(focused_1d, cmap='Greys')
plt.colorbar()
plt.show()